In [1]:
# !pip3.8 uninstall trl -y
# !pip3.8 install git+https://github.com/malaysia-ai/trl

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)

from trl import SFTTrainer

2023-08-16 09:50:42.941237: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 09:50:43.042475: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-16 09:50:43.599210: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-16 09:50:43.599274: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [3]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', trust_remote_code=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"
max_seq_length = 1536

In [4]:
tokenizer.eos_token

'</s>'

In [5]:
tokenizer.pad_token_id

0

In [6]:
!head -n 1000 shuf-combine-1536.jsonl > example.jsonl

In [7]:
from trl.trainer.utils import ConstantLengthDataset
from datasets import load_dataset
import random

In [8]:
tokenizer.eos_token

'</s>'

In [9]:
system_prompts = [
    'You are an AI assistant',
    'Anda adalah pembantu AI',
    'Anda adalah pembantu AI yang berguna',
    'You are a helpful assistant',
    'Anda seorang pembantu yang berguna',
]


def generate_and_tokenize_prompt(row):
    system_prompt = random.choice(system_prompts)
    texts = [f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n']

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                human = splitted[i].split('<manusia>:')[1]
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human)
            outputs.append(bot)
    else:
        inputs = [row['input']]
        outputs = [row['output']]
    for input, output in zip(inputs, outputs):
        texts.append(f'{input} [/INST] {output} </s><s>[INST] ')
    return {'text': ''.join(texts)}


dataset = load_dataset("json", data_files='example.jsonl', split="train")
dataset = dataset.map(generate_and_tokenize_prompt)
dataset = dataset.remove_columns(['prompt_input', 'input', 'output'])

Using custom data configuration default-95db109a2d6ceab2


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/husein/.cache/huggingface/datasets/json/default-95db109a2d6ceab2/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1000 [00:00<?, ?ex/s]

In [10]:
dataset['text'][28]

'<s>[INST] <<SYS>>\nAnda adalah pembantu AI\n<</SYS>>\n\n Saya sentiasa mahu belajar bermain gitar. Di manakah saya harus bermula?\n [/INST]  Tempat terbaik untuk bermula ialah dengan mempelajari asas teori muzik, seperti skala dan kord. Anda boleh menemui banyak sumber dalam talian yang mengajar konsep ini secara langkah demi langkah. Selain itu, anda mungkin ingin mempertimbangkan untuk mengambil pelajaran daripada pengajar berpengalaman yang boleh membantu anda mengembangkan teknik dan memperhalusi gaya permainan anda. </s><s>[INST] '

In [11]:
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=script_args.max_seq_length,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=script_args.packing,
# )

In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
# dataset_text_field = 'text'
# use_formatting_func = None
# max_seq_len = max_seq_length
# def tokenize(element):
#     outputs = tokenizer(
#         element[dataset_text_field] if not use_formatting_func else formatting_func(element),
#         truncation=True,
#         padding=False,
#         max_length=max_seq_len,
#         return_overflowing_tokens=False,
#         return_length=False,
#     )

#     if use_formatting_func and not self._dataset_sanity_checked:
#         if not isinstance(formatting_func(element), list):
#             raise ValueError(
#                 "The `formatting_func` should return a list of processed strings since it can lead to silent bugs."
#             )
#         else:
#             self._dataset_sanity_checked = True

#     return {"input_ids": outputs["input_ids"], "attention_mask": outputs["attention_mask"]}

In [15]:
from trl.trainer.utils import ConstantLengthDataset

In [34]:
constant = ConstantLengthDataset(
    tokenizer,
    dataset,
    dataset_text_field='text',
    formatting_func=None,
    seq_length=max_seq_length,
    infinite=True,
    num_of_sequences=1024,
    eos_token_id=tokenizer.eos_token_id,
)
i = iter(constant)

In [25]:
b = [next(i), next(i)]

In [26]:
b = data_collator(b)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [49]:
tokenizer.decode(next(i)['input_ids'].numpy().tolist())

'tinggal di Mongolia zaman moden.\n </s><s> [INST]  Adakah dia bekerja dengan sesiapa sahaja?\n [/INST]  Pada tahun 2009, dia dan anak perempuannya Amina menterjemah "The Diary of a Young Girl oleh Anne Frank" ke dalam bahasa Mongolia.\n </s><s> [INST]  Apakah kerja lain yang dia lakukan?\n [/INST]  Pada tahun yang sama dia menerbitkan koleksi biografi tentang penulis wanita terkenal "World\'s Best Women Writers".\n </s><s> [INST]  Adakah dia mempunyai karya lain?\n [/INST]  Shuudertsetseg kemudian beralih kepada penulisan fiksyen sejarah dan pada tahun 2010 menerbitkan "The Legendary Queen Anu" (The Legendary Queen Anu) tentang ratu Mongol Dzungar Khanate abad ke-17 yang eponim yang mengetuai tentera dan meninggal dunia pada Pertempuran Zuunmod pada tahun 1696.\n </s><s> [INST]  Apakah yang dia selidiki?\n [/INST]  Karya itu menyentuh tema yang terdapat dalam karya terdahulunya, termasuk kepentingan keluarga, pemerkasaan wanita dan identiti nasional.\n </s><s> [INST]  Berdasarkan perk